In [15]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [16]:
L, W = 1.0, 0.5  # Length and width of the domain (in cm)
lambda_ = 5.0e9  # Elastic constant (Pa)
mu = 5.0e9       # Shear modulus (Pa)
h = 1.0          # Thickness (cm)

In [17]:
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 2)
        )

    def forward(self, x, y):
        inputs = torch.cat([x, y], dim=1)
        return self.net(inputs)

In [18]:
def strain_tensor(u_x, u_y, x, y):
    u_x_x = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), retain_graph=True, create_graph=True)[0]
    u_y_y = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), retain_graph=True, create_graph=True)[0]
    u_x_y = torch.autograd.grad(u_x, y, grad_outputs=torch.ones_like(u_x), retain_graph=True, create_graph=True)[0]
    u_y_x = torch.autograd.grad(u_y, x, grad_outputs=torch.ones_like(u_y), retain_graph=True, create_graph=True)[0]

    E_xx = u_x_x
    E_yy = u_y_y
    E_xy = 0.5 * (u_x_y + u_y_x)
    return E_xx, E_yy, E_xy

In [19]:
def stress_tensor(E_xx, E_yy, E_xy):
    trace_E = E_xx + E_yy
    sigma_xx = h * (lambda_ * trace_E + 2 * mu * E_xx)
    sigma_yy = h * (lambda_ * trace_E + 2 * mu * E_yy)
    sigma_xy = h * (2 * mu * E_xy)
    return sigma_xx, sigma_yy, sigma_xy

In [20]:
def physics_loss(model, x, y):
    x.requires_grad_(True)
    y.requires_grad_(True)
    u = model(x, y)
    u_x, u_y = u[:, 0:1], u[:, 1:2]

    E_xx, E_yy, E_xy = strain_tensor(u_x, u_y, x, y)
    sigma_xx, sigma_yy, sigma_xy = stress_tensor(E_xx, E_yy, E_xy)

    sigma_xx_x = torch.autograd.grad(sigma_xx, x, grad_outputs=torch.ones_like(sigma_xx), retain_graph=True, create_graph=True)[0]
    sigma_xy_y = torch.autograd.grad(sigma_xy, y, grad_outputs=torch.ones_like(sigma_xy), retain_graph=True, create_graph=True)[0]
    sigma_yy_y = torch.autograd.grad(sigma_yy, y, grad_outputs=torch.ones_like(sigma_yy), retain_graph=True, create_graph=True)[0]
    sigma_xy_x = torch.autograd.grad(sigma_xy, x, grad_outputs=torch.ones_like(sigma_xy), retain_graph=True, create_graph=True)[0]

    residual_x = sigma_xx_x + sigma_xy_y
    residual_y = sigma_yy_y + sigma_xy_x

    loss_equilibrium = torch.mean(residual_x**2 + residual_y**2)
    return loss_equilibrium

In [21]:
def boundary_condition_loss(model, L, W):
    # Boundary A: u_x = 0, u_y = 0 at x = -L/2
    y_A = torch.linspace(-W / 2, W / 2, 100).reshape(-1, 1).requires_grad_()
    u_A = model(-L / 2 * torch.ones_like(y_A), y_A)
    loss_A = torch.mean(u_A**2)

    # Boundary D: u_x = 0.025 * L, u_y = 0 at x = L/2
    y_D = torch.linspace(-W / 2, W / 2, 100).reshape(-1, 1).requires_grad_()
    u_D = model(L / 2 * torch.ones_like(y_D), y_D)
    loss_D = torch.mean((u_D[:, 1:2]**2) + (u_D[:, 0:1] - 0.025 * L)**2)

    # Boundary B: traction-free (σ_xx = σ_xy = 0) at y = W/2
    x_B = torch.linspace(-L / 2, L / 2, 100).reshape(-1, 1).requires_grad_()
    u_B = model(x_B, W / 2 * torch.ones_like(x_B))
    u_B_x, u_B_y = u_B[:, 0:1], u_B[:, 1:2]
    E_xx_B, E_yy_B, E_xy_B = strain_tensor(u_B_x, u_B_y, x_B, W / 2 * torch.ones_like(x_B))
    sigma_xx_B, sigma_yy_B, sigma_xy_B = stress_tensor(E_xx_B, E_yy_B, E_xy_B)
    loss_B = torch.mean(sigma_xx_B**2 + sigma_xy_B**2)

    # Boundary C: traction-free (σ_xx = σ_xy = 0) at y = -W/2
    x_C = torch.linspace(-L / 2, L / 2, 100).reshape(-1, 1).requires_grad_()
    u_C = model(x_C, -W / 2 * torch.ones_like(x_C))
    u_C_x, u_C_y = u_C[:, 0:1], u_C[:, 1:2]
    E_xx_C, E_yy_C, E_xy_C = strain_tensor(u_C_x, u_C_y, x_C, -W / 2 * torch.ones_like(x_C))
    sigma_xx_C, sigma_yy_C, sigma_xy_C = stress_tensor(E_xx_C, E_yy_C, E_xy_C)
    loss_C = torch.mean(sigma_xx_C**2 + sigma_xy_C**2)

    return loss_A + loss_D + loss_B + loss_C

In [22]:
def train_pinn(model, optimizer, n_epochs, n_points, L, W):
    loss_history = []

    for epoch in range(n_epochs):
        # Sample points inside the domain
        x = torch.rand((n_points, 1)) * L - L / 2
        y = torch.rand((n_points, 1)) * W - W / 2

        loss_pde = physics_loss(model, x, y)
        loss_bc = boundary_condition_loss(model, L, W)
        loss = loss_pde + loss_bc

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())

        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.6f}")

    return loss_history

In [23]:
model = PINN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
n_epochs = 10000
n_points = 1000
loss_history = train_pinn(model, optimizer, n_epochs, n_points, L, W)


Epoch 0, Loss: 1567791980778356736.000000
Epoch 500, Loss: 56803098361856.000000
Epoch 1000, Loss: 16506323730432.000000
Epoch 1500, Loss: 8445418799104.000000
Epoch 2000, Loss: 5298567249920.000000
Epoch 2500, Loss: 4086950526976.000000
Epoch 3000, Loss: 3530405707776.000000
Epoch 3500, Loss: 6436942774272.000000
Epoch 4000, Loss: 1874246631424.000000
Epoch 4500, Loss: 1477950046208.000000
Epoch 5000, Loss: 7529187191554048.000000
Epoch 5500, Loss: 570156580864.000000
Epoch 6000, Loss: 387004891136.000000
Epoch 6500, Loss: 322169208832.000000
Epoch 7000, Loss: 117146793279488.000000
Epoch 7500, Loss: 73428279230464.000000
Epoch 8000, Loss: 487275003904.000000
Epoch 8500, Loss: 141383811072.000000
Epoch 9000, Loss: 68992483328.000000
Epoch 9500, Loss: 42430885888.000000
